In [1]:
import math
import matplotlib.pyplot as plt
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
df=pd.read_csv("/content/ACI.csv",parse_dates=['DATE'])
df.head()

#       DATE TRADING CODE   LTP*  ...    YCP  TRADE  VALUE (mn)  VOLUME
0  439 2018-05-08          ACI  375.0  ...  361.8    453      13.478  36,659
1  438 2018-06-08          ACI  373.0  ...  370.6    759      25.894  69,069
2  437 2018-07-08          ACI  373.6  ...  373.8    368      11.013  29,387
3  436 2018-08-08          ACI  369.8  ...  373.0    359       9.561  25,798
4  435 2018-09-08          ACI  365.4  ...  369.1    461      15.324  41,913

[5 rows x 12 columns]

In [2]:
# Sort DataFrame by date
df = df.sort_values('DATE')

# Double check the result
df.head()

#       DATE TRADING CODE   LTP*  ...    YCP  TRADE  VALUE (mn)  VOLUME
36  403 2018-01-10          ACI  344.5  ...  340.0    270       6.432  18,549
59  380 2018-01-11          ACI  350.4  ...  349.6    328      10.607  30,276
37  402 2018-02-10          ACI  347.5  ...  345.3    199       3.682  10,600
79  360 2018-02-12          ACI  322.2  ...  325.2    294       5.220  16,130
16  423 2018-03-09          ACI  364.0  ...  366.5    437      13.092  35,969

[5 rows x 12 columns]

In [3]:
df = df.set_index('DATE')
df.head(3)

# TRADING CODE   LTP*   HIGH  ...    YCP  TRADE  VALUE (mn)  VOLUME
DATE                                        ...                                  
2018-01-10  403          ACI  344.5  352.8  ...  340.0    270       6.432  18,549
2018-01-11  380          ACI  350.4  351.6  ...  349.6    328      10.607  30,276
2018-02-10  402          ACI  347.5  349.0  ...  345.3    199       3.682  10,600

[3 rows x 11 columns]

In [4]:
df['daily_ret'] = df['CLOSEP*'].pct_change()
df['daily_ret'] = df['daily_ret'].fillna(0.0)

In [5]:
df['volatility']= df['daily_ret'].rolling(window=21).std()
df['volatility'] = df['volatility'].fillna(0.0)

In [6]:
df['volatility'] = df['volatility']*100

In [7]:
import matplotlib.pyplot as plt 

In [8]:
df=df.reset_index()['volatility']

In [9]:
df.shape

(439,)

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))

In [11]:
##splitting dataset into train and test split where train=70 percent and test=30 percent
training_size=int(len(df)*0.70)
test_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]




In [12]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [],[]
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [13]:

# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)

In [14]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
import xgboost as xgb
from sklearn.datasets import make_regression

In [15]:
model1 = RandomForestRegressor(n_estimators=1000)
model2 =AdaBoostRegressor(n_estimators=1000,learning_rate=0.01)
model3= GradientBoostingRegressor(n_estimators=1000,learning_rate=0.01)
model4 = xgb.XGBRegressor(n_estimators=1000,learning_rate=0.01)



In [16]:
model1.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [17]:
model2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=0.01, loss='linear',
                  n_estimators=1000, random_state=None)

In [18]:
model3.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.01, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=1000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [19]:
model4.fit(X_train, y_train)

[13:16:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [20]:

train_predict1=model1.predict(X_train)
test_predict1= model1.predict(X_test)

In [21]:

train_predict2=model2.predict(X_train)
test_predict2= model2.predict(X_test)

In [22]:

train_predict3=model3.predict(X_train)
test_predict3= model3.predict(X_test)

In [23]:

train_predict4=model4.predict(X_train)
test_predict4= model4.predict(X_test)

In [24]:
ytest

array([0.28367114, 0.21379886, 0.20763901, 0.1702758 , 0.16276553,
       0.1584907 , 0.15837001, 0.15822192, 0.16912667, 0.17625208,
       0.18041492, 0.18467332, 0.1842036 , 0.37062997, 0.37037881,
       0.37784127, 0.38369051, 0.38390788, 0.38040596, 0.37895565,
       0.40476499, 0.4187    , 0.45820337, 0.45846653, 0.46368015,
       0.46702246, 0.47125747, 0.47241266, 0.47578156, 0.4694344 ,
       0.4717825 ])

In [25]:
Y_test = np.reshape(ytest,(1, ytest.size))

In [26]:
Y_test =scaler.inverse_transform(Y_test)

In [ ]:
Y_test

array([[4.8415822 , 3.64903095, 3.54389713, 2.90619719, 2.77801506,
        2.70505406, 2.7029941 , 2.70046665, 2.88658429, 3.00819794,
        3.07924753, 3.15192822, 3.14391121, 6.32575965, 6.32147298,
        6.44883925, 6.54867157, 6.5523816 , 6.49261228, 6.46785892,
        6.9083622 , 7.14619919, 7.82042649, 7.82491791, 7.91390179,
        7.97094707, 8.04322845, 8.06294477, 8.12044363, 8.01211293,
        8.05218934]])

In [27]:
test_predict1

array([0.37820511, 0.3710421 , 0.36919546, 0.37776093, 0.38719924,
       0.3898613 , 0.39736067, 0.3947177 , 0.39570986, 0.39445653,
       0.3983066 , 0.4054113 , 0.40699569, 0.40875481, 0.41921136,
       0.4162675 , 0.4170976 , 0.43247788, 0.43164334, 0.42053986,
       0.41840317, 0.44263268, 0.4400322 , 0.4514169 , 0.45448714,
       0.4576398 , 0.46664494, 0.4719579 , 0.47229311, 0.47424537,
       0.46912436])

In [28]:
test_predict1 = np.reshape(test_predict1,(1, test_predict1.size))
test_predict2 = np.reshape(test_predict2,(1, test_predict2.size))
test_predict3 = np.reshape(test_predict3,(1, test_predict3.size))
test_predict4 = np.reshape(test_predict4,(1, test_predict3.size))



In [29]:
test_predict1 =scaler.inverse_transform(test_predict1)
test_predict2 =scaler.inverse_transform(test_predict2)
test_predict3 =scaler.inverse_transform(test_predict3)
test_predict4 =scaler.inverse_transform(test_predict4)

In [30]:
test_predict1

array([[6.45504904, 6.3327938 , 6.30127611, 6.44746791, 6.60855716,
        6.65399209, 6.7819883 , 6.7368792 , 6.75381296, 6.73242161,
        6.79813293, 6.91939318, 6.94643474, 6.97645881, 7.15492676,
        7.10468217, 7.11885005, 7.38135426, 7.36711068, 7.17760109,
        7.14113292, 7.55467231, 7.51028841, 7.70459769, 7.75699926,
        7.8108076 , 7.96450372, 8.05518311, 8.06090426, 8.09422465,
        8.00682129]])

In [31]:
test_predict2

array([[6.96056351, 6.61440553, 6.72246609, 6.60159579, 6.78500751,
        7.00311761, 7.53931084, 7.5171713 , 7.47299098, 7.46801885,
        7.48686839, 7.5171713 , 7.5077159 , 7.49357385, 7.55550854,
        7.55575244, 7.57781367, 7.62463405, 7.62921301, 7.62168629,
        7.57781367, 7.59068404, 7.56725718, 7.6737927 , 7.67808382,
        7.70154128, 7.71228987, 7.7799406 , 7.79111211, 7.8204271 ,
        7.68704552]])

In [32]:
test_predict3

array([[6.30937048, 6.1238205 , 6.00963211, 6.15855202, 6.47055576,
        6.54741898, 6.52984213, 6.78127055, 6.92744612, 7.22806125,
        7.24434795, 7.31070813, 7.39576494, 7.41282412, 7.49310746,
        7.49321871, 7.48857046, 7.8692006 , 7.89180306, 7.01937363,
        7.05857195, 7.40429074, 7.44441556, 7.98813842, 8.03023123,
        7.86739833, 8.18059812, 8.46314049, 8.46314049, 8.43381874,
        8.17590854]])

In [33]:
test_predict4

array([[6.7938576, 6.594382 , 6.219163 , 6.198428 , 6.6875143, 6.8370748,
        6.953405 , 7.0505466, 7.0574183, 7.0637045, 7.0413675, 6.902991 ,
        6.9921203, 7.0207047, 7.0822034, 7.01919  , 7.0235267, 7.4362006,
        7.4440413, 7.1489677, 7.1368313, 7.5153203, 7.521453 , 8.412553 ,
        8.492106 , 8.50168  , 8.599469 , 8.822075 , 8.826375 , 8.823448 ,
        8.590627 ]], dtype=float32)

In [34]:
#Y_test = Y_test.transpose()
RMSE1 = math.sqrt(mean_squared_error(Y_test,test_predict1))
RMSE1

2.332695324353076

In [35]:
#Y_test = Y_test.transpose()
RMSE2 = math.sqrt(mean_squared_error(Y_test,test_predict2))
RMSE2

2.705347502850653

In [36]:
#Y_test = Y_test.transpose()
RMSE3 = math.sqrt(mean_squared_error(Y_test,test_predict3))
RMSE3

2.4149142490513746

In [37]:
#Y_test = Y_test.transpose()
RMSE4 = math.sqrt(mean_squared_error(Y_test,test_predict4))
RMSE4

2.4467483069561293

In [38]:

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict1)
print('MAE: %f' % mae)

MAE: 1.682475


In [39]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict2)
print('MAE: %f' % mae)

MAE: 2.049143


In [40]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict3)
print('MAE: %f' % mae)

MAE: 1.807848


In [41]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test,test_predict4)
print('MAE: %f' % mae)

MAE: 1.879881
